In [1]:
import spacy
import pandas as pd
import os
import sys






In [ ]:
BASE_PATH = os.path.abspath(os.path.join('..')) # base path of project

file_path = BASE_PATH + '/data/raw/amazon_reviews_us_Electronics_v1_00.tsv'

raw_data = pd.read_table(file_path,error_bad_lines=False, nrows=100)

#review_body = raw_data['review_body']
reviews = raw_data[['review_id', 'review_body']]
nlp=spacy.load("en_core_web_lg")

In [2]:
BASE_PATH = os.getcwd()
PARENT = os.path.dirname(BASE_PATH)
os.chdir(PARENT)
#sys.path.insert(0,PARENT)
from src.models import aspect_extraction
print(os.getcwd())

/Users/Ishika/MSA_Courses/CSE_6242/Project/TheSavageArray


In [29]:
def apply_extraction(row,nlp):
    review_body = row['review_body']
    review_id = row['review_id']

    doc=nlp(review_body)


    ## FIRST RULE OF DEPENDANCY PARSE -
    ## M - Sentiment modifier || A - Aspect
    ## RULE = M is child of A with a relationshio of amod
    rule1_pairs = []
    for token in doc:
        if token.dep_ == "amod":
            rule1_pairs.append((token.head.text, token.text))
            #return row['height'] * row['width']


    ## SECOND RULE OF DEPENDANCY PARSE -
    ## M - Sentiment modifier || A - Aspect
    #Direct Object - A is a child of something with relationship of nsubj, while
    # M is a child of the same something with relationship of dobj
    #Assumption - A verb will have only one NSUBJ and DOBJ

    rule2_pairs = []
    for token in doc:
        children = token.children
        A = "999999"
        M = "999999"
        for child in children :
            if(child.dep_ == "nsubj"):
                A = child.text
            if(child.dep_ == "dobj"):
                M = child.text
        if(A != "999999" and M != "999999"):
            rule2_pairs.append((A, M))


    ## THIRD RULE OF DEPENDANCY PARSE -
    ## M - Sentiment modifier || A - Aspect
    #Adjectival Complement - A is a child of something with relationship of nsubj, while
    # M is a child of the same something with relationship of acomp
    #Assumption - A verb will have only one NSUBJ and DOBJ

    rule3_pairs = []

    for token in doc:

        children = token.children
        A = "999999"
        M = "999999"
        for child in children :
            if(child.dep_ == "nsubj"):
                A = child.text

            if(child.dep_ == "acomp"):
                M = child.text

        if(A != "999999" and M != "999999"):
            rule3_pairs.append((A, M))

    ## FOURTH RULE OF DEPENDANCY PARSE -
    ## M - Sentiment modifier || A - Aspect

    #Adverbial modifier to a passive verb - A is a child of something with relationship of nsubjpass, while
    # M is a child of the same something with relationship of advmod

    #Assumption - A verb will have only one NSUBJ and DOBJ

    rule4_pairs = []
    for token in doc:


        children = token.children
        A = "999999"
        M = "999999"
        for child in children :
            if(child.dep_ == "nsubjpass"):
                A = child.text

            if(child.dep_ == "advmod"):
                M = child.text

        if(A != "999999" and M != "999999"):
            rule4_pairs.append((A, M))


    ## FIFTH RULE OF DEPENDANCY PARSE -
    ## M - Sentiment modifier || A - Aspect

    #Complement of a copular verb - A is a child of M with relationship of nsubj, while
    # M has a child with relationship of cop

    #Assumption - A verb will have only one NSUBJ and DOBJ

    rule5_pairs = []
    for token in doc:
        children = token.children
        A = "999999"
        buf_var = "999999"
        for child in children :
            if(child.dep_ == "nsubj"):
                A = child.text

            if(child.dep_ == "cop"):
                buf_var = child.text

        if(A != "999999" and buf_var != "999999"):
            rule3_pairs.append((A, token.text))

    aspects = []
    aspects = rule1_pairs + rule2_pairs + rule3_pairs +rule4_pairs +rule5_pairs
    dic = {"review_id" : review_id , "aspect_pairs" : aspects}
    return dic


In [32]:
review_decomp = reviews.apply(lambda row: apply_extraction(row,nlp), axis=1)


In [33]:
#from gensim.models import Word2Vec
import spacy
#from spacy import displacy
from sklearn import cluster 
from collections import defaultdict

#from src/models import aspect_extraction

In [51]:
NUM_CLUSTERS = 4

In [5]:
# review_decomp = [{"aspect_pairs": [("sound", "great"),("music", "loud"),("shipping", "poor")]},
#                  {"aspect_pairs": [("packaging", "pathetic"),("sound", "perfect")]},
#                  {"aspect_pairs": [("quality", "wonderful"),("photos", "great")]}]

In [35]:
#nlp = spacy.load('en_core_web_lg')  # make sure to use larger model!
#tokens = nlp(u'sound music shipping packaging bass')
#for token1 in tokens:
#    for token2 in tokens:
  #      print(token1.text, token2.text, token1.similarity(token2))

In [3]:
# change this when running from main script
review_decomp = aspect_extraction.aspect_extraction()
# nlp = spacy.load('en_core_web_lg') 

FileNotFoundError: File b'/Users/Ishika/MSA_Courses/CSE_6242/Project/data/raw/amazon_reviews_us_Electronics_v1_00.tsv' does not exist

In [60]:
aspects = []
for review in review_decomp:
    aspect_pairs = review["aspect_pairs"]
    for noun, adj in aspect_pairs:
        print(noun,adj)
        aspects.append(noun)

unique_aspects = list(set(aspects)) # use this list for clustering

Bass lacking
quality good
five more
I stars
sound Great
Alll good
gain Excellent
price great
speaker small
volume loud
volume great
hdmi micro
machines.<br other
wait&#34;.<br protected
It s
I machine
that collection
lip flat
stability best
lip short
piece front
grip firmest
idea best
style executive
place nifty
reach easy
location cumbersome
layer good
quality top
you lip
I it
it chair
I it
I turn
/>*I product
I able
/>It constructed
price Great
product great
TV upstairs
I it
product Good
br old
style new
way Fantastic
charge fast
tape sticky
I usb
I this
speakers etc
I it
it <
product great
service great
device Great
device little
HDMI micro
hdmi full
cables.<br new
mine far
mine />Its
sound Excellent
price Good
plastic expensive
/>1884 Ministerial
34;You money
you choice.&#34
You Money
they uncomfortable
headphones comfortable
sound better
charging compact
product convenient
charger slimmest
service Courteous
I it
I happy
solution Great
fine fit
This expensive
Works great
sound enou

In [40]:
len(unique_aspects)

226

In [37]:
# need this mapping later for tagging clusters
aspects_map = defaultdict(int) 
for asp in aspects:
    aspects_map[asp] += 1

In [41]:
#nlp = spacy.load('en_core_web_lg')  # make sure to use larger model!

asp_vectors = []
for aspect in unique_aspects:
    print(aspect)
    token = nlp(aspect)
    asp_vectors.append(token.vector)

customers
Something
system
pair
stability
reduction
family
Battery
surface
punch
Bass
they
TV
tips
things
min
weeks
case
genres
clarity
amplifier
battery
stations
friend
HDMI
cables.<br
song
characteristic
wait&#34;.<br
who
feature
company
highs
We
You
style
five
time
side
load
caller
shipping
she
inconsistency
br
tapping
most
volume
mode
34;You
bonus
issue
/>You
flights
reach
trial
cable
receiver
achievement
speaker
speakers
types
service
audio
stories
pieces
colors
traveler
sound
hookup
design
reviews
Walkman
guitar
ports
response
hdmi
gift
mine
something
/>*I
headset
bass
malfunction
this
problem
noise
you
tape
members
line
/>Computer
looking
channels
microphone
latter
charging
muddle.<br
review.<br
industry
setup.<br
which
road
features
location
device
sound.<br
show
models
idea
world
layer
designer
flight
months
networks
mount
people
job
hand
investment
i
Flare
He
everything
instructions
IEMs
gain
2000s
profile
buy
lip
She
chance
charge
They
products
it
son
router
packaging
year
q

In [52]:
kmeans = cluster.KMeans(n_clusters=NUM_CLUSTERS)
kmeans.fit(asp_vectors)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=4, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [53]:
#centroids = kmeans.cluster_centers_
labels = kmeans.labels_ 
print ("Cluster id labels for inputted data")
#print (unique_aspects)
#print (labels)

asp_to_cluster_map = dict(zip(unique_aspects,labels))
print(asp_to_cluster_map)


Cluster id labels for inputted data
{'customers': 2, 'Something': 1, 'system': 2, 'pair': 0, 'stability': 2, 'reduction': 2, 'family': 1, 'Battery': 3, 'surface': 2, 'punch': 0, 'Bass': 3, 'they': 1, 'TV': 0, 'tips': 0, 'things': 1, 'min': 0, 'weeks': 0, 'case': 2, 'genres': 0, 'clarity': 2, 'amplifier': 3, 'battery': 3, 'stations': 2, 'friend': 1, 'HDMI': 3, 'cables.<br': 0, 'song': 1, 'characteristic': 2, 'wait&#34;.<br': 0, 'who': 1, 'feature': 2, 'company': 2, 'highs': 0, 'We': 1, 'You': 1, 'style': 2, 'five': 1, 'time': 1, 'side': 1, 'load': 2, 'caller': 0, 'shipping': 0, 'she': 1, 'inconsistency': 0, 'br': 0, 'tapping': 0, 'most': 1, 'volume': 2, 'mode': 2, '34;You': 0, 'bonus': 0, 'issue': 1, '/>You': 0, 'flights': 0, 'reach': 1, 'trial': 0, 'cable': 3, 'receiver': 3, 'achievement': 0, 'speaker': 3, 'speakers': 3, 'types': 2, 'service': 2, 'audio': 3, 'stories': 1, 'pieces': 1, 'colors': 2, 'traveler': 0, 'sound': 3, 'hookup': 3, 'design': 2, 'reviews': 0, 'Walkman': 3, 'guitar'

In [58]:
cluster_map = defaultdict()
cluster_to_asp_map = defaultdict()
for i in range(NUM_CLUSTERS):
    cluster_nouns = [k for k,v in asp_to_cluster_map.items() if v == i]
#     print(cluster_nouns)
    freq_map = {k:v for k,v in aspects_map.items() if k in cluster_nouns}
    freq_map = sorted(freq_map.items(), key = lambda x: x[1], reverse = True)
#     print(freq_map)
    cluster_map[i] = freq_map[0][0]
    cluster_to_asp_map[i] = cluster_nouns

In [59]:
print(cluster_to_asp_map)

defaultdict(None, {0: ['pair', 'punch', 'TV', 'tips', 'min', 'weeks', 'genres', 'cables.<br', 'wait&#34;.<br', 'highs', 'caller', 'shipping', 'inconsistency', 'br', 'tapping', '34;You', 'bonus', '/>You', 'flights', 'trial', 'achievement', 'traveler', 'reviews', 'guitar', 'gift', '/>*I', 'malfunction', 'tape', '/>Computer', 'muddle.<br', 'review.<br', 'setup.<br', 'sound.<br', 'designer', 'flight', 'months', 'Flare', 'instructions', '2000s', 'buy', 'lip', 'review', 'box', '/>I', '24', 'machines.<br', 'bargain', 'mid', 'set', 'albums', 'jam', '/>1884', 'Alll', '/>It', 'American', 'gadget', 'FLAC', 'price', 'R2s', 'notes', 'ezcast', 'slick!!<br', 'heaviness', 'Instructions', 'hours', '10', 'recipient', 'lows'], 1: ['Something', 'family', 'they', 'things', 'friend', 'song', 'who', 'We', 'You', 'five', 'time', 'side', 'she', 'most', 'issue', 'reach', 'stories', 'pieces', 'mine', 'something', 'this', 'problem', 'you', 'members', 'looking', 'latter', 'which', 'road', 'show', 'idea', 'world', 

In [56]:
print(cluster_map)

defaultdict(None, {0: '/>I', 1: 'I', 2: 'quality', 3: 'sound'})
